In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
def summary_stats_table(data):
    '''
    a functino to summerize all types of data
    '''
    # count of nulls
    missing_counts = pd.DataFrame(data.isnull().sum())
    missing_counts.columns = ['count_null']

    # numeric column stats
    num_stats = data.select_dtypes(include=['int64','float64']).describe().loc[['count','min','max']].transpose()
    num_stats['dtype'] = data.select_dtypes(include=['int64','float64']).dtypes.tolist()

    # non-numeric value stats
    non_num_stats = data.select_dtypes(exclude=['int64','float64']).describe().transpose()
    non_num_stats['dtype'] = data.select_dtypes(exclude=['int64','float64']).dtypes.tolist()
    non_num_stats = non_num_stats.rename(columns={"first": "min", "last": "max"})

    # merge all 
    stats_merge = pd.concat([num_stats, non_num_stats], axis=0, join='outer', ignore_index=False, keys=None,
              levels=None, names=None, verify_integrity=False, copy=True).fillna("").sort_values('dtype')

    column_order = ['dtype', 'count', 'count_null','unique','min','max','top','freq']
    summary_stats = pd.merge(stats_merge, missing_counts, left_index=True, right_index=True)[column_order]
    return(summary_stats)

In [3]:
import numpy as np  # linear algebra
import pandas as pd  #
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge,Lasso,ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.svm import SVR, LinearSVC
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import os

ModuleNotFoundError: No module named 'mlxtend'

In [5]:
conda install mlxtend


Note: you may need to restart the kernel to use updated packages.



CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/win-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

ConnectTimeout(MaxRetryError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Max retries exceeded with url: /pkgs/main/win-64/current_repodata.json (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x00000000063AD6A0>, 'Connection to repo.anaconda.com timed out. (connect timeout=9.15)'))"))




In [4]:
# feature engineering 作者：KeithFish
# GitHub：KeithFish （https://github.com/KeithFish）
# 模型部分， 借用blend思路， 不用融合别人的预测


#######################################################数据导入和特征提取-【开始】################################################################################
#显示当前编译器的Draft Environment下的文件；将文件夹下的对应名称csv文件储存为矩阵对象。
#print(os.listdir("../input"))
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

#显示矩阵对象的维数，核查是否导入成功
print("Train set size:", train.shape)
print("Test set size:", test.shape)

#记录运算开始的时间
print('START data processing', datetime.now(), )

#取出train矩阵（或者称之为“数据帧”dataframe）中title为'Id'的列，赋值给train_ID。所以train_ID是一维列向量了。test_ID类似。
#train_ID和test_ID根本就没有使用，完全可以删除。
#train_ID = train['Id']
#test_ID = test['Id']

##################删除训练集和测试集中的标签列-【开始】#################
#将train矩阵中的'Id'列删除（原地删除，故将inplace设为true），因为原始数据中的数据索引和预测模型的构建没有关系。
#test矩阵类似。
train.drop(['Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)
##################删除训练集和测试集中的标签列-【结束】#################

###############删除训练集中的极端值和进行数据index更新-【开始】#########
#使用条件筛选操作，通过覆值的方式剔除原始数据train矩阵中的极端值（极端值也被称为outliers），帮助预防房价预测模型出现过拟合。剔除操作也可以视为前剪枝。
train = train[train.GrLivArea < 4500]
#由于删去了部分行，故此时train矩阵中的index列并不连续。使用reset_index命令，在固定非index数据的顺序的前提下（inplace=True），重新对index编号（drop=True）。
train.reset_index(drop=True, inplace=True)
###############删除训练集中的极端值和进行数据index更新-【结束】#########

##########对预测目标数值进行对数变换和特征矩阵对象的创建-【开始】#######
# log1p就是log(1+x)，用来对房价数据进行数据预处理，它的好处是转化后的数据更加服从高斯分布，有利于后续的分类结果。
# 需要注意，最后需要将预测出的平滑数据还原，而还原过程就是log1p的逆运算expm1。
train["SalePrice"] = np.log1p(train["SalePrice"])
#单独取出训练数据中的房价列信息，存入y对象。
#y = train.SalePrice.reset_index(drop=True) #.reset_index(drop=True)方法：在原有的索引列重置索引，不再另外添加新列。有必要使用reset_index吗？有必要的，不这样做y将有两套index，作为df的y将有两列。
y = train['SalePrice'].reset_index(drop=True) #对上式的改写
#沿着水平的方向寻找列名为'SalePrice'的列（们），把它们对应的列统统删掉。得到了单纯的特征矩阵，存入train_features对象中。
train_features = train.drop(['SalePrice'], axis=1)
#test本来就没有房价列，所以它本来就是单纯的特征矩阵。
test_features = test
##########对预测目标数值进行对数变换和特征矩阵对象的创建-【结束】#######

##合并训练数据特征矩阵与测试数据特征矩阵，以便统一进行特征处理-【开始】##
#将训练数据中的特征矩阵和测试数据中的特征矩阵合并（.concat[矩阵1,矩阵2]），并对合并后的矩阵index重新编号（.reset_index(drop=True)）。
features = pd.concat([train_features, test_features]).reset_index(drop=True)
#检查合并后的矩阵的维数，核查合并结果。
print("剔除训练数据中的极端值后，将其特征矩阵和测试数据中的特征矩阵合并，维度为:",features.shape)
##合并训练数据特征矩阵与测试数据特征矩阵，以便统一进行特征处理-【结束】##
#######################################################数据导入和特征提取-【结束】################################################################################

##############################################################特征处理-【开始】###################################################################################
#对于列名为'MSSubClass'、'YrSold'、'MoSold'的特征列，将列中的数据类型转化为string格式。
features['MSSubClass'] = features['MSSubClass'].apply(str)
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

###############################填充空值-【开始】##########################
#按照以下各个特征列的实际情况，依次处理各个特征列中的空值（.fillna()方法）
features['Functional'] = features['Functional'].fillna('Typ') #空值填充为str型数据'Typ'
features['Electrical'] = features['Electrical'].fillna("SBrkr") #空值填充为str型数据"SBrkr"
features['KitchenQual'] = features['KitchenQual'].fillna("TA") #空值填充为str型数据"TA"
features["PoolQC"] = features["PoolQC"].fillna("None") #空值填充为str型数据"None"

#对于列名为'Exterior1st'、'Exterior2nd'、'SaleType'的特征列，使用列中的众数填充空值。
#	1.先查找数据列中的众数：使用df.mode()[]方法
#	  解释：df.mode(0或1,0表示对列查找，1表示对行查找)[需要查找众数的df列的index（就是df中的第几列）]，将返回数据列中的众数
#	2.使用.fillna()方法进行填充
features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0]) 
features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

#对于列名为'GarageYrBlt', 'GarageArea', 'GarageCars'的特征列，使用0填充空值。
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    features[col] = features[col].fillna(0)

#对于列名为'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'的特征列，使用字符串'None'填充空值。
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    features[col] = features[col].fillna('None')

#对于列名为'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'的特征列，使用字符串'None'填充空值。
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('None')

#聚合函数（按某一列关键字分组）groupby，它的特点是：将返回与传入方法的矩阵维度相同的单个序列。
#transform是与groupby（pandas中最有用的操作之一）通常组合使用，它对传入方法的矩阵进行维度不变的变换。具体变换方法写在括号中，通常会使用匿名函数，对传入矩阵的所有元素进行操作。
#对于features矩阵，按照'MSSubClass'列中的元素分布进行分组，被分组的数据列是'MSZoning'列。feature.groupby(被作为索引的列的名称)[被分组的数据列的名称]
#features.groupby('MSSubClass')['MSZoning']后，得到的是一个以'MSSubClass'为索引，以'MSZoning'为数据列的矩阵。
#.transform()方法将对'MSZoning'数据列进行()内的变换，它将返回和传入矩阵同样维度的矩阵。
#括号内是匿名函数，将对传入矩阵中的空值进行填充，使用的填充元素是传入矩阵中的众数。
features['MSZoning'] = features.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))


#判断出features矩阵中列为对象的列，将列名存入objects叔祖。对于features矩阵中的各个列对象，将其列中的空值填充为'None'
objects = []
for i in features.columns:
    if features[i].dtype == object:
        objects.append(i)
features.update(features[objects].fillna('None'))

#使用传入矩阵（'LotFrontage'列）的中位数对传入矩阵中的空值进行填充。
#先以'Neighborhood'为标签，以'LotFrontage'为被汇总序列。然后使用被汇总序列中的中位数，对原始矩阵'LotFrontage'列中的空值进行填充。
#transform的特性是同维操作，最后输出结果的顺序和原始数据在序号上完全匹配。
features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

#对于整型和浮点型数据列，使用0填充其中的空值。
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics.append(i)
features.update(features[numerics].fillna(0))
###############################填充空值-【结束】##########################

######################数字型数据列偏度校正-【开始】#######################
#使用skew()方法，计算所有整型和浮点型数据列中，数据分布的偏度（skewness）。
#偏度是统计数据分布偏斜方向和程度的度量，是统计数据分布非对称程度的数字特征。亦称偏态、偏态系数。 
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics2.append(i)
skew_features = features[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

#以0.5作为基准，统计偏度超过此数值的高偏度分布数据列，获取这些数据列的index。
high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

#对高偏度数据进行处理，将其转化为正态分布。
#Box和Cox提出的变换可以使线性回归模型满足线性性、独立性、方差齐次以及正态性的同时，又不丢失信息。
for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))#这是boxcox1p的使用方法，参数的具体意义暂时不解释
######################数字型数据列偏度校正-【结束】#######################

Train set size: (1460, 81)
Test set size: (1459, 80)
START data processing 2020-01-05 09:34:53.996979
剔除训练数据中的极端值后，将其特征矩阵和测试数据中的特征矩阵合并，维度为: (2917, 79)


In [5]:
######################特征删除和融合创建新特征-【开始】###################
#删除一些特征。df.drop（‘列名’, axis=1）代表将‘列名’对应的列标签（们）沿着水平的方向依次删掉。
features = features.drop(['Utilities', 'Street', 'PoolQC',], axis=1)

# features = features.drop(['MSSubClass', 'Street', 'PoolQC', 'LandSlope', 
#                           'Exterior1st', 'Exterior2nd', 'BsmtFinType1', 
#                           'BsmtFinType2', 'Electrical', 'Functional'], axis=1)

In [6]:
#融合多个特征，生成新特征。
features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']
features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']

features['Total_sqr_footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                 features['1stFlrSF'] + features['2ndFlrSF'])

features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                               features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))

features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
                              features['EnclosedPorch'] + features['ScreenPorch'] +
                              features['WoodDeckSF'])

#added by Alex
features['yearslived'] = pd.to_numeric(features['YrSold']) - features['YearBuilt']
features['yearsremod'] = pd.to_numeric(features['YrSold']) - features['YearRemodAdd']

#简化numeric特征。对于某些分布单调（比如100个数据中有99个的数值是0.9，另1个是0.1）的数字型数据列，进行01取值处理。
features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
features['has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

#检查特征处理后，特征矩阵的维数，核查特征处理结果。
print("删除了3个特征，又融合创建了12个新特征，处理之后的特征矩阵维度为:",features.shape)
######################特征删除和融合创建新特征-【结束】###################

删除了3个特征，又融合创建了12个新特征，处理之后的特征矩阵维度为: (2917, 88)


In [7]:
# double check the feature distribution before the one-hot-encoding
feature_summary = summary_stats_table(features)
feature_summary[feature_summary.dtype == 'object'].sort_values(by='unique', ascending=False)
# feature_summary.loc[ ['YrBltAndRemod' , 'yearsremod', 'yearslived', 'YrSold'] , : ]

,dtype,count,count_null,unique,min,max,top,freq
Neighborhood,object,2917.0,0,25,,,NAmes,443
Exterior2nd,object,2917.0,0,16,,,VinylSd,1015
MSSubClass,object,2917.0,0,16,,,20,1079
Exterior1st,object,2917.0,0,15,,,VinylSd,1026
MoSold,object,2917.0,0,12,,,6,503
SaleType,object,2917.0,0,9,,,WD,2526
Condition1,object,2917.0,0,9,,,Norm,2511
Condition2,object,2917.0,0,8,,,Norm,2888
HouseStyle,object,2917.0,0,8,,,1Story,1471
GarageType,object,2917.0,0,7,,,Attchd,1722


In [8]:
# for MI based feature selection
features_cat = features.iloc[:len(y), :]
outliers = [30, 88, 462, 631, 1322]
features_cat = features_cat.drop(features_cat.index[outliers])
features_cat.shape

(1453, 88)

In [9]:
####################特征投影、特征矩阵拆解和截取-【开始】#################
#使用.get_dummies()方法对特征矩阵进行类似“坐标投影”操作。获得在新空间下的特征表达。
#get_dummies is one-hot encoding for categorical variables
final_features = pd.get_dummies(features).reset_index(drop=True)

#打印新空间下的特征维数，也是新空间的维数。
print("使用get_dummies()方法“投影”特征矩阵，即分解出更多特征，得到更多列。投影后的特征矩阵维度为:",final_features.shape)

#进行特征空间降阶。截取前len(y)行，存入X阵（因为之前进行了训练数据和测试数据的合并，所以从合并矩阵中取出前len(y)行，就得到了训练数据集的处理后的特征矩阵）。
#截取剩余部分，即从序号为len(y)的行开始，至矩阵结尾的各行，存入X_sub阵。此为完成特征变换后的测试集特征矩阵。
#注：len(df)是行计数方法
X = final_features.iloc[:len(y), :]	#y是列向量，存储了训练数据中的房价列信息。截取后得到的X阵的维度是len(y)*(final_features的列数)。
X_sub = final_features.iloc[len(y):, :]#使用len命令，求矩阵X的长度，得到的是矩阵对象的长度，即有矩阵中有多少列，而不是每列上有多少行。

#在新生特征空间中，剔除X阵和y阵中有着极端值的各行数据（因为X和y阵在水平方向上是一致的，所以要一起删除同样的行）。outliers数值中给出了极端值的列序号。
#df.drop(df.index[序号])将删除指定序号的各行。再使用=对df覆值。
outliers = [30, 88, 462, 631, 1322]
X = X.drop(X.index[outliers])#因为X阵是经过对特征矩阵进行类似“坐标投影”操作后得到的，列向量y中的行号对应着X阵中的列号。
y = y.drop(y.index[outliers])
####################特征投影、特征矩阵拆解和截取-【结束】#################

print("删除了3个特征，又融合创建了12个新特征，处理之后的特征矩阵维度为:",'X', X.shape, 'y', y.shape, 'X_sub', X_sub.shape)

使用get_dummies()方法“投影”特征矩阵，即分解出更多特征，得到更多列。投影后的特征矩阵维度为: (2917, 335)
删除了3个特征，又融合创建了12个新特征，处理之后的特征矩阵维度为: X (1453, 335) y (1453,) X_sub (1459, 335)


In [10]:
#############个体机器学习模型的创建（即模型声明和参数设置）-【开始】############
alphas_alt = [14.5, 14.6,  14.8, 15,  15.2, 15.3,  15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

from sklearn.linear_model import LassoLarsCV
lassolarsVote = LassoLarsCV(cv = 10)
from sklearn.linear_model import LarsCV
larsVote = LarsCV(cv = 10)

ridgeVote = RidgeCV(alphas=alphas_alt, cv=10)

lassoVote = LassoCV(max_iter=1e7, alphas=alphas2,
                              random_state=42, cv=10)# Lasso(alpha = 0.0004)

elasticnetVote = ElasticNetCV(max_iter=1e7, alphas=e_alphas,
                                        cv=10, l1_ratio=e_l1ratio)

svrVote = SVR(kernel = 'linear',C= 20, epsilon= 0.008, gamma=0.0003,)

#定义random forest model									
rf = RandomForestRegressor(n_estimators=3000, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, criterion='mse', random_state =42)                             

#定义GB梯度提升模型（展开到一阶导数）									
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)                             

#定义lightgbm模型									
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                         random_state =23,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )

#定义xgboost模型（展开到二阶导数）                                      
xgboostVote = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=4,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
#############个体机器学习模型的创建（即模型声明和参数设置）-【结束】############

In [11]:
def get_top_n_features(train_data_X, train_data_y, top_n_features,models = []):
        # 随机森林
    #定义xgboost模型（展开到二阶导数）  
    features_top_n = []
    all_features = list(train_data_X)
    for model in models:
        

        model.fit(train_data_X, train_data_y)
        #将feature按Importance排序
        try:
            feature_imp_sorted_each = pd.DataFrame({'feature': all_features, 'importance': list(abs(model.coef_))}).sort_values('importance', ascending=False)
            sort_list = list(abs(model.coef_))
            sort_list.sort(reverse = True)
            print(sort_list)
        except:
            feature_imp_sorted_each = pd.DataFrame({'feature': all_features, 'importance': list(abs(model.feature_importances_))}).sort_values('importance', ascending=False)
            sort_list = list(abs(model.feature_importances_))
            sort_list.sort(reverse = True)
            print(sort_list)
        features_top_n_each = feature_imp_sorted_each.head(top_n_features)['feature']
        print('Sample 5 Features from Classifier')
        print(str(features_top_n_each[:5]))
        if(len(features_top_n) == 0):
            features_top_n = features_top_n_each
        features_top_n = pd.concat([features_top_n,features_top_n_each], ignore_index=True).drop_duplicates()
#        feature_imp_sorted_each.to_csv("lars_importance.csv", index=True)
    
    for item in features_top_n.items():
         all_features.remove(str(item[1]))
    removed_fetures = all_features
    print("删掉的FEATURE:",removed_fetures)
    print("选取的FEATURE个数",len(features_top_n))
    
    return features_top_n

In [12]:
feature_to_pick = 214
models = [lightgbm]#,lightgbm,gbr,xgboost]
features_top_n = get_top_n_features(X,y,feature_to_pick,models = models)
X = X[features_top_n]
X_sub = X_sub[features_top_n]

[727, 643, 544, 476, 474, 472, 450, 405, 404, 402, 374, 358, 345, 344, 342, 340, 336, 316, 299, 241, 183, 182, 167, 161, 154, 153, 147, 137, 132, 123, 117, 116, 112, 112, 110, 110, 105, 101, 98, 90, 90, 84, 82, 81, 79, 78, 77, 75, 67, 66, 64, 64, 64, 62, 62, 61, 59, 58, 57, 55, 54, 51, 51, 50, 50, 47, 46, 45, 44, 44, 43, 40, 39, 37, 37, 37, 36, 36, 36, 35, 35, 34, 34, 34, 34, 32, 31, 30, 29, 29, 29, 28, 28, 28, 28, 27, 27, 26, 25, 25, 25, 25, 24, 24, 24, 24, 23, 23, 23, 23, 23, 22, 22, 21, 21, 21, 20, 20, 20, 19, 19, 19, 18, 18, 18, 17, 17, 17, 17, 16, 16, 16, 16, 16, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 13, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 9, 9, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

['LowQualFinSF', 'BsmtHalfBath', 'has2ndfloor', 'hasbsmt', 'MSSubClass_150', 'MSSubClass_160', 'MSSubClass_180', 'MSSubClass_190', 'MSSubClass_40', 'MSSubClass_45', 'MSSubClass_50', 'MSSubClass_75', 'MSSubClass_80', 'MSSubClass_85', 'MSZoning_FV', 'MSZoning_RH', 'Alley_None', 'LotShape_IR2', 'LotShape_IR3', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_CollgCr', 'Neighborhood_Gilbert', 'Neighborhood_MeadowV', 'Neighborhood_NPkVill', 'Neighborhood_SWISU', 'Neighborhood_SawyerW', 'Neighborhood_Timber', 'Neighborhood_Veenker', 'Condition1_Feedr', 'Condition1_PosN', 'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Artery', 'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'BldgType_2fmCon', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_SFoyer', 'RoofStyle_Gambrel', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'RoofMatl_CompShg', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_CBlock', 'Exterior1st_HdBoard', 'Exterior1st_ImStucc', 'Exterior1st_Plywood', 'Exterior1st_Stone', 'Exterior1st_Stucco', 'Exterior1st_WdShing', 'Exterior2nd_AsbShng', 'Exterior2nd_AsphShn', 'Exterior2nd_BrkFace', 'Exterior2nd_CBlock', 'Exterior2nd_CmentBd', 'Exterior2nd_HdBoard', 'Exterior2nd_ImStucc', 'Exterior2nd_MetalSd', 'Exterior2nd_Plywood', 'Exterior2nd_Stone', 'Exterior2nd_Stucco', 'Exterior2nd_VinylSd', 'MasVnrType_BrkCmn', 'ExterCond_Fa', 'ExterCond_Gd', 'ExterCond_Po', 'Foundation_Slab', 'Foundation_Stone', 'BsmtQual_Fa', 'BsmtQual_None', 'BsmtCond_Gd', 'BsmtCond_Po', 'BsmtExposure_Av', 'BsmtExposure_Mn', 'BsmtFinType1_BLQ', 'BsmtFinType1_LwQ', 'BsmtFinType1_Rec', 'BsmtFinType2_ALQ', 'BsmtFinType2_BLQ', 'BsmtFinType2_GLQ', 'BsmtFinType2_LwQ', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'KitchenQual_Fa', 'Functional_Maj2', 'Functional_Mod', 'FireplaceQu_Ex', 'GarageType_2Types', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageFinish_RFn', 'GarageQual_Ex', 'GarageQual_Gd', 'GarageQual_Po', 'GarageCond_Ex', 'GarageCond_Fa', 'GarageCond_Gd', 'GarageCond_Po', 'Fence_GdPrv', 'Fence_MnPrv', 'Fence_MnWw', 'MiscFeature_Gar2', 'MiscFeature_None', 'MiscFeature_Othr', 'MiscFeature_Shed', 'MiscFeature_TenC', 'MoSold_1', 'MoSold_10', 'MoSold_3', 'MoSold_4', 'MoSold_8', 'YrSold_2007', 'YrSold_2008', 'YrSold_2010', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_Oth', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family']

In [13]:
##############################################################机器学习-【开始】###################################################################################
print('特征处理已经完成。开始对训练数据进行机器学习', datetime.now())

#设置k折交叉验证的参数。
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)


#定义均方根对数误差（Root Mean Squared Logarithmic Error ，RMSLE）
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


#创建模型评分函数，根据不同模型的表现打分
#cv表示Cross-validation,交叉验证的意思。
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

#ADD by Alex, for stack cv score print
def cv_rmse_stack(model, X=np.array(X),y = np.array(y) ):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error",cv = kfolds,n_jobs=4))
    return (rmse)

特征处理已经完成。开始对训练数据进行机器学习 2020-01-05 09:34:55.493609


In [14]:
# setup models    
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=kfolds))

ridge_meta = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=kfolds))

lasso = make_pipeline(RobustScaler(),
                      LassoCV(max_iter=1e7, alphas=alphas2,
                              random_state=42, cv=kfolds))

elasticnet = make_pipeline(RobustScaler(),
                           ElasticNetCV(max_iter=1e7, alphas=e_alphas,
                                        cv=kfolds, l1_ratio=e_l1ratio))
                                        
svr = make_pipeline(RobustScaler(),
                      SVR(C= 20, epsilon= 0.008, gamma=0.0003,))


gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
                                   

lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )
                                       

xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

# stack
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet,
                                            gbr,xgboost, lightgbm),
                                meta_regressor=ridge_meta,
                                use_features_in_secondary=True)
                                

print('TEST score on CV')

score = cv_rmse(ridge)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(svr)
print("SVR score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lightgbm)
print("Lightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(gbr)
print("GradientBoosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(xgboost)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

#time consuming , run with local GPU for 15min, more then 30min on kaggle kernel CPU 
#score = cv_rmse_stack(stack_gen)
#print("Stack_final score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

baseline performence
TEST score on CV
Kernel Ridge score: 0.1011 (0.0126)
 2019-12-30 15:15:03.104934
Lasso score: 0.0996 (0.0131)
 2019-12-30 15:15:09.400107
ElasticNet score: 0.0997 (0.0131)
 2019-12-30 15:15:32.878642
SVR score: 0.1017 (0.0115)
 2019-12-30 15:15:39.310478
Lightgbm score: 0.1074 (0.0143)
 2019-12-30 15:15:46.552786
GradientBoosting score: 0.1089 (0.0127)
 2019-12-30 15:16:56.147644
Xgboost score: 0.1048 (0.0149)
 2019-12-30 15:18:04.934837
Stack_final score: 0.1009 (0.0126)
 2019-12-30 15:31:10.783572

In [15]:
print('START Fit')
print(datetime.now(), 'StackingCVRegressor')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))
print(datetime.now(), 'elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)
print(datetime.now(), 'lasso')
lasso_model_full_data = lasso.fit(X, y)
print(datetime.now(), 'ridge')
ridge_model_full_data = ridge.fit(X, y)
print(datetime.now(), 'svr')
svr_model_full_data = svr.fit(X, y)
print(datetime.now(), 'GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)
print(datetime.now(), 'xgboost')
xgb_model_full_data = xgboost.fit(X, y)
print(datetime.now(), 'lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

START Fit
2020-01-05 09:34:55.517704 StackingCVRegressor
2020-01-05 09:36:31.373366 elasticnet
2020-01-05 09:36:33.028397 lasso
2020-01-05 09:36:33.448032 ridge
2020-01-05 09:36:34.230144 svr
2020-01-05 09:36:34.839546 GradientBoosting
2020-01-05 09:36:42.372148 xgboost
2020-01-05 09:36:48.482155 lightgbm


In [16]:
def blend_models_predict(X):
    return ((0.10 * elastic_model_full_data.predict(X)) + \
            (0.10 * lasso_model_full_data.predict(X)) + \
            (0.1 * ridge_model_full_data.predict(X)) + \
            (0.10 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.1 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.3 * stack_gen_model.predict(np.array(X))))
            
print('RMSLE score on train data:')

# need to be higher than 0.05785573801347548 to be better
print(rmsle(y, blend_models_predict(X)))

RMSLE score on train data:
0.07076561431354544


#RMSLE score on train data:
#0.060216506203416176

In [17]:
########将测试集的特征矩阵作为输入，传入训练好的模型，得出的输出写入.csv文件的第2列-【开始】########
print('使用测试集特征进行房价预测 Predict submission', datetime.now(),)
submission = pd.read_csv("house-prices-advanced-regression-techniques/sample_submission.csv")
#函数注释：.iloc[:,1]是基于索引位来选取数据集，[索引1:索引2]，左闭右开。
submission.iloc[:,1] = np.floor(np.expm1(blend_models_predict(X_sub)))
########将测试集的特征矩阵作为输入，传入训练好的模型，得出的输出写入.csv文件的第2列-【结束】########

#---------------------------------------------------------------------------------------------------------------------------#
#---------------------------------------------------------------------------------------------------------------------------#
#---------------------------------------------------------------------------------------------------------------------------#

# # 当对平台未公开的测试集进行预测时，前述模型的误差是0.114 this kernel gave a score 0.114
# # 为了提高模型的得分，引入其他模型的优秀预测结果，与前述模型的预测结果进行混合（有点类似抄别人的答案，但实质是扩大集成模型的规模，引入更多的模型） let's up it by mixing with the top kernels

# ######################模型输出结果融合-【开始】#############################
# #在多模型集成学习器预测结果的基础上，融合其他优秀模型（即平台上其他均方根对数误差小的模型）的预测结果。
# #这步操作是为了降低多模型集成学习器的方差。
# print('融合其他优秀模型的预测结果 Blend with Top Kernals submissions', datetime.now(),)
# sub_1 = pd.read_csv('../input/top-10-0-10943-stacking-mice-and-brutal-force/House_Prices_submit.csv')
# sub_2 = pd.read_csv('../input/hybrid-svm-benchmark-approach-0-11180-lb-top-2/hybrid_solution.csv')
# sub_3 = pd.read_csv('../input/lasso-model-for-regression-problem/lasso_sol22_Median.csv')
# submission.iloc[:,1] = np.floor((0.25 * np.floor(np.expm1(blend_models_predict(X_sub)))) + 
#                                 (0.25 * sub_1.iloc[:,1]) + 
#                                 (0.25 * sub_2.iloc[:,1]) + 
#                                 (0.25 * sub_3.iloc[:,1]))
# ######################模型输出结果融合-【结束】#############################      

# ####################融合结果的极端值剔除-【开始】###########################  
# #处理融合后结果中的极端值。把太大的数值（降序排列时，位于顶部往下0.005的数值，就是只有0.005的数比它大）缩小一点（乘以0.77），把太小的数值（降序排列时，位于顶部往下0.99的数值）放大一点（乘以1.1）
# q1 = submission['SalePrice'].quantile(0.005)
# q2 = submission['SalePrice'].quantile(0.995)
# submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x > q1 else x*0.77)
# submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x < q2 else x*1.1)
# ####################融合结果的极端值剔除-【结束】###########################


from datetime import date, datetime
d = date.today().strftime("%Y_%b_%d_")
t = datetime.now().strftime("%H_%M")

#以csv文件的形式输出预测值
submission.to_csv("House_price_submission_evah_{}.csv".format(d+t), index=False)
print('融合结果.csv文件输出成功 Save submission', datetime.now())
##############################################################机器学习-【结束】###################################################################################

使用测试集特征进行房价预测 Predict submission 2020-01-05 09:36:49.992435
融合结果.csv文件输出成功 Save submission 2020-01-05 09:36:50.793641


In [18]:
# # clean up the output csv files
# import os
# import glob
# for dirname, _, filenames in os.walk('/kaggle/working/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# files = glob.glob('/kaggle/working/*.csv')
# for f in files:
#     os.remove(f)
    
# print(files)

score = cv_rmse_stack(stack_gen)

print("Stack_final score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [19]:
golden = pd.read_csv('house-prices-advanced-regression-techniques/golden.csv')
def submit(model,test):
    score = rmsle(np.log(np.expm1(model.predict(test))),np.log(golden.iloc[:,1]))
    return score
for model in [elastic_model_full_data,lasso_model_full_data,ridge_model_full_data,svr_model_full_data,gbr_model_full_data,xgb_model_full_data,lgb_model_full_data]:
    print(submit(model,X_sub))

print(submit(stack_gen_model,np.array(X_sub)))
print("final:",rmsle(np.log(submission.iloc[:,1]),np.log(golden.iloc[:,1])))

0.12336205725622867
0.12327624275684786
0.12359734460161327
0.1292810757790663
0.12691270044810168
0.12407763219526512
0.1237011919181776
0.12143036820293235
final: 0.12105712976513625


In [20]:
import pandas_profiling as ppf
report = ppf.ProfileReport(features)
report.to_file('profile_report.html')